In [1]:
import pathlib
import tensorflow as tf
import numpy as np
import os
import keras
import json
import cv2
from matplotlib import pyplot as plt
import zipfile

"""
This function will load the data and return two zipped arrays:

output:       contains an image stored as a numpy array
class_labels: contains the corresponding label for each element of output

@params
dir: should be the path that contains both train images, and json dic with labels
save: If you want it to save the output as a zip file (so you don't have to do this twice)
"""

def load_data(dir, save = False):

    #labels is a dictionary mapping file_numer ->class_label
    with open(dir + "/train_gt.json") as file:
        labels = json.load(file)

    #converts an image to a numpy array

    """
    Eventually we will want to resize the images when we localize. This is just so that they are all the same size.
    """
    get_image = lambda file_name: cv2.resize(cv2.imread(file_name),(40, 100))

    output = []
    class_labels = []

    #iterate through the folders, convert the images to RGB arrays, and then append the class label
    for folder in list(os.listdir(dir+"/images")):

        if folder == '.DS_Store': continue
        #if (folder != '1' and folder != '2'): continue

        cls = labels[folder]
        images = os.listdir(os.path.join(dir+"/images", folder))

        for image in images:
            output.append(get_image(os.path.join(dir+"/images", folder, image)))
            class_labels.append(cls)

    zip_file = zip(output, class_labels)
    if save:
        np.savez_compressed(dir+"/numpy_data.npz", output, labels)

    return zip_file

def load_small_data(dir):

    data = np.load(dir)
    lst = data.files
    X = data[lst[0]]
    y = data[lst[1]]

    return X,y

In [4]:
X,y = load_small_data('/content/jersey_sample.npz')

### **Processing**

In [2]:
from keras import Sequential
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, ZeroPadding2D
from sklearn.model_selection import train_test_split

In [3]:
# right now, there is no data processing.
# But, We need to make sure that all the images have the same shape.
# and one hot encode y labels
def process_data(X_data, y_data):

    """
    Prepare the response:

    This part of the function just basically maps the classes to the set {0, ... 44}, and then one-hot encodes the response
    """
    #for to_categorical to work, we need to map the labels to {0, ... 44}
    label_to_int = {}
    int_to_label = {}

    for index, label in enumerate(np.unique(y_data)):
        label_to_int[label]=index
        int_to_label[index]=label


    y_data_new = np.vectorize(label_to_int.__getitem__)(y_data)
    y_data_new = to_categorical(y_data_new)

    #X_train, y_train = zip(*load_data(directory, True))
    X_data = np.array(X_data)


    return X_data, y_data_new

In [5]:
X_train, y_train = process_data(X,y)

y_train.shape

(36650, 45)

### **Model with data augmentation**

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Create an ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,  # Random rotations
    width_shift_range=0.1,  # Random horizontal shifts
    height_shift_range=0.1,  # Random vertical shifts
    zoom_range=0.1,  # Random zoom
    horizontal_flip=True,  # Random horizontal flips
    fill_mode='nearest'  # Strategy for filling in newly created pixels
)

###### TODO: change to gray scale
###### INV

# Define the input shape
input_shape = (100, 40, 3)

# Define the model
model = Sequential()

# First convolutional block
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second convolutional block
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third convolutional block
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Fully connected block
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(45, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 38, 32)        896       
                                                                 
 activation (Activation)     (None, 98, 38, 32)        0         
                                                                 
 batch_normalization (Batch  (None, 98, 38, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 19, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 49, 19, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 17, 64)        1

In [ ]:
model.fit(data_generator.flow(X_train, y_train), epochs=20)

Epoch 1/20
1146/1146 [==============================] - 424s 367ms/step - loss: 2.8094 - accuracy: 0.2627
Epoch 2/20
1146/1146 [==============================] - 416s 363ms/step - loss: 2.1378 - accuracy: 0.3642
Epoch 3/20
1146/1146 [==============================] - 411s 359ms/step - loss: 1.9024 - accuracy: 0.4249
Epoch 4/20
1146/1146 [==============================] - 425s 371ms/step - loss: 1.7497 - accuracy: 0.4662
Epoch 5/20
1146/1146 [==============================] - 416s 363ms/step - loss: 1.6450 - accuracy: 0.4938
Epoch 6/20
1146/1146 [==============================] - 436s 380ms/step - loss: 1.5670 - accuracy: 0.5169
Epoch 7/20
1146/1146 [==============================] - 435s 379ms/step - loss: 1.5049 - accuracy: 0.5317
Epoch 8/20
1146/1146 [==============================] - 454s 396ms/step - loss: 1.4445 - accuracy: 0.5510
Epoch 9/20
1146/1146 [==============================] - 439s 383ms/step - loss: 1.4106 - accuracy: 0.5603
Epoch 10/20
1146/1146 [=======================